In [ ]:
# Dependencies.
from bs4 import BeautifulSoup as bs
import os
import pandas as pd

In [ ]:
# Remove all shows before 2000, as many questions are outdated.
# In the future I might keep these to recommend answers in the Doc2Vec stage, but not serve the clue in the web app.
deleteList = []
for filename in os.listdir("j-archive"):
    with open(f"j-archive\{filename}",encoding="utf8") as file:
        html = file.read()
    soup = bs(html, "lxml")
    airdate = soup.title.text.split("aired ")[-1]
    year = int(airdate.split("-")[0])
    if year<2000:
        deleteList.append(filename)
for filename in deleteList:
    os.remove(f"j-archive\{filename}")

In [ ]:
# Remove all shows with no rounds (not sure why these exist).
noneList = []
for filename in os.listdir("j-archive"):
    with open(f"j-archive\{filename}",encoding="utf8") as file:
        html = file.read()
    soup = bs(html, "lxml")
    round1 = soup.find("div",id="jeopardy_round")
    round2 = soup.find("div",id="double_jeopardy_round")
    round3 = soup.find("div",id="final_jeopardy_round")
    if round1 is None and round2 is None and round3 is None:
        noneList.append(filename)
for filename in noneList:
    os.remove(f"j-archive\{filename}")

In [ ]:
# Scan through all the files, gather all clue and category data.
categoryID = 0
clueID = 0
filenameList = []
categoryIDList = []
clueIDList = []
roundList = []
categoryList = []
categoryCommentList = []
valueList = []
clueList = []
answerList = []
for filename in os.listdir("j-archive"):
    with open(f"j-archive\{filename}",encoding="utf8") as file:
        html = file.read()
    soup = bs(html, "lxml")
    # Check for the existance of each round (in the csvs tiebreaker is included with FJ round as there are very few instances).
    round1 = soup.find("div",id="jeopardy_round")
    round2 = soup.find("div",id="double_jeopardy_round")
    round3 = soup.find("div",id="final_jeopardy_round")
    roundTB = soup.find("var",id="clue_TB_stuck")
    if round1 is not None:
        for categoryIndex in range(6):
            for clueIndex in range(5):
                # Value of a clue is based on position in the table, not the "clue_value" class as it changes on daily doubles.
                if soup.find("td",id=f"clue_J_{categoryIndex+1}_{clueIndex+1}") is not None:
                    # Ignore all clues with links to media.
                    if soup.find("td",id=f"clue_J_{categoryIndex+1}_{clueIndex+1}").find("a") is None:
                        # For convenience, the show is identified by filename until the very end, when it is converted to showID.
                        filenameList.append(filename)
                        categoryIDList.append(categoryID)
                        clueIDList.append(clueID)
                        roundList.append(1)
                        categoryList.append(soup.find("div",id="jeopardy_round").find_all("td",class_="category_name")[categoryIndex].get_text(separator=" "))
                        categoryCommentList.append(soup.find("div",id="jeopardy_round").find_all("td",class_="category_comments")[categoryIndex].get_text(separator=" "))
                        valueList.append((clueIndex+1)*200)
                        clueList.append(soup.find("td",id=f"clue_J_{categoryIndex+1}_{clueIndex+1}").get_text(separator=" "))
                        answerList.append(" ".join(bs(soup.find("td",id=f"clue_J_{categoryIndex+1}_{clueIndex+1}").parent.parent.find("div")["onmouseover"],"lxml").find("em").get_text(separator=" ")).split())
                        clueID += 1
            categoryID += 1
    if round2 is not None:
        for categoryIndex in range(6):
            for clueIndex in range(5):
                if soup.find("td",id=f"clue_DJ_{categoryIndex+1}_{clueIndex+1}") is not None:
                    if soup.find("td",id=f"clue_DJ_{categoryIndex+1}_{clueIndex+1}").find("a") is None:
                        filenameList.append(filename)
                        categoryIDList.append(categoryID)
                        clueIDList.append(clueID)
                        roundList.append(2)
                        categoryList.append(soup.find("div",id="double_jeopardy_round").find_all("td",class_="category_name")[categoryIndex].get_text(separator=" "))
                        categoryCommentList.append(soup.find("div",id="double_jeopardy_round").find_all("td",class_="category_comments")[categoryIndex].get_text(separator=" "))
                        valueList.append((clueIndex+1)*400)
                        clueList.append(soup.find("td",id=f"clue_DJ_{categoryIndex+1}_{clueIndex+1}").get_text(separator=" "))
                        answerList.append(" ".join(bs(soup.find("td",id=f"clue_DJ_{categoryIndex+1}_{clueIndex+1}").parent.parent.find("div")["onmouseover"],"lxml").find("em").get_text(separator=" ")).split())
                        clueID += 1
            categoryID += 1
    if round3 is not None:
        if soup.find("td",id="clue_FJ").find("a") is None:
            # FJ is given a value of 0, TB a value of -1 to maintain integer type.
            filenameList.append(filename)
            categoryIDList.append(categoryID)
            clueIDList.append(clueID)
            roundList.append(3)
            categoryList.append(soup.find("div",id="final_jeopardy_round").find_all("td",class_="category_name")[0].get_text(separator=" "))
            categoryCommentList.append(soup.find("div",id="final_jeopardy_round").find_all("td",class_="category_comments")[0].get_text(separator=" "))
            valueList.append(0)
            clueList.append(soup.find("td",id="clue_FJ").get_text(separator=" "))
            answerList.append(" ".join(bs(soup.find("div",id="final_jeopardy_round").find_all("td",class_="category")[0].find("div")["onmouseover"],"lxml").find("em").get_text(separator=" ")).split())
            clueID += 1
            categoryID += 1
    if roundTB is not None:
        if soup.find("td",id="clue_TB").find("a") is None:
            filenameList.append(filename)
            categoryIDList.append(categoryID)
            clueIDList.append(clueID)
            roundList.append(3)
            categoryList.append(soup.find("div",id="final_jeopardy_round").find_all("td",class_="category_name")[1].get_text(separator=" "))
            categoryCommentList.append(soup.find("div",id="final_jeopardy_round").find_all("td",class_="category_comments")[1].get_text(separator=" "))
            valueList.append(-1)
            clueList.append(soup.find("td",id="clue_TB").get_text(separator=" "))
            answerList.append(" ".join(bs(soup.find("div",id="final_jeopardy_round").find_all("td",class_="category")[1].find("div")["onmouseover"],"lxml").find("em").get_text(separator=" ")).split())
            clueID += 1
            categoryID += 1
# Build dataframe to use pandas for further analysis.
clueDf = pd.DataFrame()
clueDf['filename'] = filenameList
clueDf['categoryID'] = categoryIDList
clueDf['clueID'] = clueIDList
clueDf['round'] = roundList
clueDf['category'] = categoryList
clueDf['categoryComment'] = categoryCommentList
clueDf['value'] = valueList
clueDf['clue'] = clueList
clueDf['answer'] = answerList
# Remove additional slashes.
clueDf["answer"] = clueDf["answer"].str.replace("\\\\'","'")
clueDf["answer"] = clueDf["answer"].str.replace("\\\\'","'")
# Some clue answers are "null" which is saved as a blank.
clueDf['answer'].fillna("Null",inplace=True)
clueDf.to_csv("clueRaw.csv")

In [ ]:
# Build documents to be used for clustering (category name, all clues, all answers).
clusterFilenameList = []
clusterCategoryList = []
clusterCategoryCommentList = []
clusterDocumentList = []
for i in clueDf['categoryID'].unique():
    clusterFilenameList.append(clueDf[clueDf['categoryID']==i]['filename'].tolist()[0])
    clusterCategoryList.append(clueDf[clueDf['categoryID']==i]['category'].tolist()[0])
    clusterCategoryCommentList.append(clueDf[clueDf['categoryID']==i]['categoryComment'].tolist()[0])
    clusterDocumentList.append(" ".join([clueDf[clueDf['categoryID']==i]['category'].tolist()[0]," ".join(clueDf[clueDf['categoryID']==i]['clue'].tolist())," ".join(clueDf[clueDf['categoryID']==i]['answer'].tolist())]))
clusteringDf = pd.DataFrame()
clusteringDf['filename'] = clusterFilenameList
clusteringDf['categoryID'] = clueDf['categoryID'].unique()
clusteringDf['category'] = clusterCategoryList
clusteringDf['categoryComment'] = clusterCategoryCommentList
clusteringDf['document'] = clusterDocumentList
clusteringDf.to_csv("clusteringRaw.csv")

In [ ]:
# Build metadata file, show subtype is determined from the jarchive comment, show type is built by hand.
metadataAirdateList = []
metadataTypeList = []
metadataSubtypeList = []
for filename in os.listdir("j-archive"):
    with open(f"j-archive\{filename}",encoding="utf8") as file:
        html = file.read()
    soup = bs(html, "lxml")
    metadataAirdateList.append(soup.title.text.split("aired ")[-1])
    comment = soup.find("div",id="game_comments").text.replace("\n"," ")
    # Order is to prevent kids week replacing kids week reunion and etc.
    if "Back to School Week" in comment:
        metadataTypeList.append("Kids")
        metadataSubtypeList.append("Back to School Week")
    elif "Kids Week Reunion" in comment:
        metadataTypeList.append("College")
        metadataSubtypeList.append("Kids Week Reunion")
    elif "Holiday Kids Week" in comment:
        metadataTypeList.append("Kids")
        metadataSubtypeList.append("Holiday Kids Week")
    elif "Kids Week" in comment:
        metadataTypeList.append("Kids")
        metadataSubtypeList.append("Kids Week")
    elif "Teen Tournament Summer Games" in comment:
        metadataTypeList.append("Teen")
        metadataSubtypeList.append("Teen Tournament Summer Games")
    elif "Teen Tournament" in comment:
        metadataTypeList.append("Teen")  
        metadataSubtypeList.append("Teen Tournament")
    elif "Celebrity Jeopardy!" in comment:
        metadataTypeList.append("Celebrity")
        metadataSubtypeList.append("Celebrity Jeopardy!")
    elif "Power Players Week" in comment:
        metadataTypeList.append("Celebrity")
        metadataSubtypeList.append("Power Players Week")
    elif "Million Dollar Celebrity Invitational" in comment:
        metadataTypeList.append("Celebrity")
        metadataSubtypeList.append("Million Dollar Celebrity Invitational")
    elif "College Championship" in comment:
        metadataTypeList.append("College")
        metadataSubtypeList.append("College Championship")
    elif "Ultimate Tournament of Champions" in comment:
        metadataTypeList.append("Champions")
        metadataSubtypeList.append("Ultimate Tournament of Champions")
    elif "Tournament of Champions" in comment:
        metadataTypeList.append("Champions")
        metadataSubtypeList.append("Tournament of Champions")
    elif "Million Dollar Masters" in comment:
        metadataTypeList.append("Champions")
        metadataSubtypeList.append("Million Dollar Masters")
    elif "The IBM Challenge" in comment:
        metadataTypeList.append("Champions")
        metadataSubtypeList.append("The IBM Challenge")
    elif "Battle of the Decades" in comment:
        metadataTypeList.append("Champions")
        metadataSubtypeList.append("Battle of the Decades")
    elif "All-Star Games" in comment:
        metadataTypeList.append("Champions")
        metadataSubtypeList.append("All-Star Games")
    elif "Jeopardy!: The Greatest of All Time" in comment:
        metadataTypeList.append("Champions")
        metadataSubtypeList.append("Jeopardy! Greatest of All Time")
    elif "International Championship" in comment:
        metadataTypeList.append("Other")
        metadataSubtypeList.append("International Championship")
    elif "Teachers Tournament" in comment:
        metadataTypeList.append("Other")
        metadataSubtypeList.append("Teachers Tournament")
    else:
        metadataTypeList.append("Regular")
        metadataSubtypeList.append("Regular")
metadataDf = pd.DataFrame()
metadataDf['filename'] = os.listdir("j-archive")
metadataDf['airdate'] = metadataAirdateList
metadataDf['type'] = metadataTypeList
metadataDf['subtype'] = metadataSubtypeList
metadataDf.index.name = "showID"
metadataDf.to_csv("metadata.csv")